In [1]:
import base64
import requests

# 🔐 Your Sandbox credentials (keep them private!)
client_id = 
client_secret = 

# Encode client ID and secret
credentials = f"{client_id}:{client_secret}"
encoded_credentials = base64.b64encode(credentials.encode()).decode()

# OAuth2 Token request
token_url = 'https://api.sandbox.ebay.com/identity/v1/oauth2/token'
headers = {
    'Authorization': f'Basic {encoded_credentials}',
    'Content-Type': 'application/x-www-form-urlencoded'
}
data = {
    'grant_type': 'client_credentials',
    'scope': 'https://api.ebay.com/oauth/api_scope/buy.marketplace.insights'
}

response = requests.post(token_url, headers=headers, data=data)

# Check and extract token
if response.status_code == 200:
    access_token = response.json()['access_token']
    print("✅ Access token retrieved successfully!")
else:
    print("❌ Failed to retrieve token")
    print("Status Code:", response.status_code)
    print("Details:", response.text)

✅ Access token retrieved successfully!


In [3]:
# 🔎 eBay Sandbox API: Get sold iPhone data
api_url = 'https://api.sandbox.ebay.com/buy/marketplace_insights/v1_beta/item_sales/search'
params = {
    'q': 'iphone',            # Keyword to search
    'category_ids': '9355',   # Category ID for smartphones
    'limit': '10'             # Number of results to fetch
}
headers = {
    'Authorization': f'Bearer {access_token}',
    'X-EBAY-C-MARKETPLACE-ID': 'EBAY_US'
}

# Send GET request to eBay API
response = requests.get(api_url, headers=headers, params=params)

# Convert JSON response
data = response.json()

In [4]:
import pandas as pd

# Extract key fields
items = []
for item in data.get('itemSales', []):
    items.append({
        'Title': item.get('title'),
        'Price': item.get('lastSoldPrice', {}).get('value'),
        'Currency': item.get('lastSoldPrice', {}).get('currency'),
        'Sold Date': item.get('lastSoldDate'),
        'Seller': item.get('seller', {}).get('username'),
        'Condition': item.get('condition'),
        'Quantity Sold': item.get('totalSoldQuantity')
    })

# Create DataFrame
df = pd.DataFrame(items)
df

""


In [5]:
df.head(10)

""


In [6]:
import json
print(json.dumps(data, indent=4))

{
    "href": "http://api.sandbox.ebay.com/buy/marketplace_insights/v1_beta/item_sales/search?q=iphone&category_ids=9355&offset=0&limit=10",
    "total": 0,
    "limit": 10,
    "offset": 0
}


In [ ]:
import requests
import base64
import pandas as pd
import json

# --- PASTE YOUR NOW-ACTIVE PRODUCTION KEYS HERE ---
CLIENT_ID_PRODUCTION = "YOUR_ACTIVE_PRODUCTION_CLIENT_ID_HERE"
CLIENT_SECRET_PRODUCTION = "YOUR_ACTIVE_PRODUCTION_CLIENT_SECRET_HERE"
# ---

def get_production_token():
    """Gets the access token from the REAL eBay production server."""
    print("1. Getting Production access token...")
    url = "https://api.ebay.com/identity/v1/oauth2/token" 
    headers = {
        "Content-Type": "application/x-www-form-urlencoded",
        "Authorization": "Basic " + base64.b64encode(f"{CLIENT_ID_PRODUCTION}:{CLIENT_SECRET_PRODUCTION}".encode()).decode(),
    }
    body = {"grant_type": "client_credentials", "scope": "https://api.ebay.com/oauth/api_scope"}
    response = requests.post(url, headers=headers, data=body)
    response.raise_for_status() 
    print("   SUCCESS! Production Token obtained.")
    return response.json()["access_token"]

def get_real_sold_data(access_token, keyword):
    """Gets sold data from the REAL eBay production server."""
    print(f"\n2. Searching for REAL sold items: '{keyword}'...")
    url = "https://api.ebay.com/buy/marketplace_insights/v1/item_sales/search"
    headers = {"Authorization": f"Bearer {access_token}", "X-EBAY-C-MARKETPLACE-ID": "EBAY_US"}
    params = {"q": keyword, "limit": 100}
    response = requests.get(url, headers=headers, params=params)
    response.raise_for_status()
    print("   SUCCESS! Real data received.")
    return response.json()


# --- MAIN PROGRAM ---
try:
    token = get_production_token()
    product_to_research = "iPhone 13" # Let's research a popular product
    sold_data = get_real_sold_data(token, product_to_research)
    
    if 'itemSales' in sold_data and sold_data['itemSales']:
        df = pd.DataFrame(sold_data['itemSales'])
        df['price'] = df['lastSoldPrice'].apply(lambda x: float(x['value']))
        average_price = df['price'].mean()
        
        print("\n3. Analysis Complete!")
        print("=================================================")
        print(f"SUCCESS! The average selling price for '{product_to_research}' is: ${average_price:.2f}")
        print("=================================================")
        
        # Show a sample of the data you received
        print("\nHere is a sample of the real data you collected:")
        print(df[['title', 'price', 'condition']].head().to_string())

except requests.exceptions.HTTPError as err:
    print(f"\n--- AN ERROR OCCURRED ---")
    print("This shouldn't happen now, but if it does, check that your keys were copied correctly.")
    print(f"HTTP Error: {err.response.status_code}")
    print(json.dumps(err.response.json(), indent=2))
